In [17]:
#!python3
# Copyright 2025 ACCESS-NRI and contributors. See the top-level COPYRIGHT file for details.
# SPDX-License-Identifier: Apache-2.0
# dependencies: module use /g/data/xp65/public/modules ; module load conda/analysis3

import xarray as xr
import numpy as np
import glob
import dask
from dask.distributed import Client
import cftime
import datetime
import netCDF4
import shutil
import os


CICE_DAILY_FN = "access-om3.cice.1day.mean.1981-??-??.nc" #"access-om3.cice.1day.mean.????-??-??-?????.nc"
# STUB_FN = "access-om3.cice.1day.mean."


def start_of_day(dt):
    """
    For a cftime.datetime object, return the start of the month
    """
    # Get the class of the input datetime (e.g., DatetimeNoLeap, Datetime360Day, etc.)
    dt_class = type(dt)
    first_day = dt_class(dt.year, dt.month, 1)

    return dt_class(first_day.year, first_day.month, first_day.day)

def monthly_ranges(start,end):
    monthly_range = xr.date_range(
        start = str(start_of_day(start)) ,
        end = str(end)  ,
        freq = 'MS' , 
        use_cftime = True,
        calendar = daily_ds.time.values[0].calendar
    )

    #add the end in
    monthly_range = [*monthly_range, end]

    return monthly_range

In [2]:
if __name__ == "__main__":

    jobfs = os.environ['PBS_JOBFS']

    # use first node only, to avoid copying files to jobfs on other nodes
    n_worker = min(
      int(os.environ['PBS_NCI_NCPUS_PER_NODE']), int(os.environ['PBS_NCPUS'])
    )

    # mem_worker = int(os.environ['PBS_VMEM'])/n_worker

In [3]:
    netCDF4.set_chunk_cache(size = 134217728/2) #128MB

In [4]:
    client = Client(threads_per_worker = 1, n_workers = n_worker)

2025-06-27 13:26:08,545 - distributed.nanny - WARNING - Restarting worker
Task exception was never retrieved
future: <Task finished name='Task-7471468' coro=<Client._gather.<locals>.wait() done, defined at /g/data/xp65/public/apps/med_conda/envs/analysis3-25.05/lib/python3.11/site-packages/distributed/client.py:2377> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.05/lib/python3.11/site-packages/distributed/client.py", line 2386, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-7471459' coro=<Client._gather.<locals>.wait() done, defined at /g/data/xp65/public/apps/med_conda/envs/analysis3-25.05/lib/python3.11/site-packages/distributed/client.py:2377> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.05/lib/python3.11/site-packages/distributed/client.py", line 2386, in wait
    r

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 104
Total threads: 104,Total memory: 496.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43935,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:38887,Total threads: 1
Dashboard: /proxy/34335/status,Memory: 4.77 GiB
Nanny: tcp://127.0.0.1:42331,


In [6]:
! rm {jobfs}/{CICE_DAILY_FN}

In [7]:
%%time
# #If no directory option provided , then use latest
        # if [ -z $out_dir ]; then 
        #     #latest output dir only
        #     out_dir=$(ls -drv archive/output*[0-9] | head -1) 
        # fi

in_dir = "/g/data/tm70/as2285/payu/peturb_tests/perturb_base_no_thermo_check/archive/output005/"

daily_f = glob.glob(f"{in_dir}/{CICE_DAILY_FN}")

for iF in daily_f:
    shutil.copy(iF, jobfs)

CPU times: user 3.74 s, sys: 9.52 s, total: 13.3 s
Wall time: 8.31 s


In [8]:
    daily_f = glob.glob(f"{jobfs}/{CICE_DAILY_FN}")

In [9]:
    daily_ds = xr.open_mfdataset(daily_f,
        compat='override', data_vars='minimal', coords='minimal', combine_attrs='override',
        parallel=True, 
        decode_times = xr.coders.CFDatetimeCoder(use_cftime=True),
    )

In [10]:
daily_ds

<xarray.Dataset> Size: 82GB
Dimensions:      (time: 365, nbnd: 2, nc: 5, nkice: 4, nksnow: 1, nkbio: 3,
                  nkaer: 5, nj: 1142, ni: 1440)
Coordinates:
  * time         (time) object 3kB 1981-01-01 12:00:00 ... 1981-12-31 12:00:00
Dimensions without coordinates: nbnd, nc, nkice, nksnow, nkbio, nkaer, nj, ni
Data variables: (12/26)
    time_bounds  (time, nbnd) object 6kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    NCAT         (nc) float64 40B dask.array<chunksize=(5,), meta=np.ndarray>
    VGRDi        (nkice) float64 32B dask.array<chunksize=(4,), meta=np.ndarray>
    VGRDs        (nksnow) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    VGRDb        (nkbio) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    VGRDa        (nkaer) float64 40B dask.array<chunksize=(5,), meta=np.ndarray>
    ...           ...
    snoice       (time, nj, ni) float64 5GB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    fhocn        (time, nj, ni) float64 5GB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    dvidtt       (time, nj, ni) float64 5GB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    dvidtd       (time, nj, ni) float64 5GB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    fsurf_ai     (time, nj, ni) float64 5GB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    sifb         (time, nj, ni) float64 5GB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
Attributes: (12/13)
    title:               access-om3
    contents:            Diagnostic and Prognostic Variables
    source:              CICE Sea Ice Model, unknown_version_name
    comment:             All years have exactly 365 days
    comment2:            File written on model date 19810102
    comment3:            seconds elapsed into model date:      0
    ...                  ...
    time_axis_position:  middle
    Conventions:         CF-1.8
    external_variables:   TLON TLAT ULON ULAT NLON NLAT ELON ELAT NFSD tmask ...
    history:             This dataset was created on 2025-06-26 at 08:52
    date_created:        2025-06-26 08:52
    io_flavor:           io_pio2 hdf5

In [18]:
    times = daily_ds.time.values

    monthly_range = monthly_ranges(np.min(times),np.max(times))
    monthly_pairs = list(zip(monthly_range[:-1],monthly_range[1:]))
    monthly_ncs = list()

In [19]:
monthly_pairs

[(cftime.DatetimeNoLeap(1981, 1, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 2, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(1981, 2, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 3, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(1981, 3, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 4, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(1981, 4, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 5, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(1981, 5, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 6, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(1981, 6, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 7, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(1981, 7, 1, 0, 0, 0, 0, has_year_zero=True),
  cftime.DatetimeNoLeap(1981, 8, 1, 0, 0, 0, 0, has_year_zero=True)),
 (cftime.DatetimeNoLeap(198

In [20]:
%%time
for iRange in monthly_pairs:
    month_ds = daily_ds.sel(time=slice(*iRange))

    month_ds.chunk({'time':len(month_ds.time), 'ni':'auto', 'nj':'auto'})

    # monthly_ncs.append(
    month_ds.to_netcdf(
        f"{jobfs}/{MONTHLY_STUB_FN}{str(iRange[0])[0:7]}.nc",
        # compute = False
    )
    # )

CPU times: user 711 ms, sys: 47.2 ms, total: 758 ms
Wall time: 756 ms


In [21]:
month_ds

<xarray.Dataset> Size: 7GB
Dimensions:      (time: 31, nbnd: 2, nc: 5, nkice: 4, nksnow: 1, nkbio: 3,
                  nkaer: 5, nj: 1142, ni: 1440)
Coordinates:
  * time         (time) object 248B 1981-12-01 12:00:00 ... 1981-12-31 12:00:00
Dimensions without coordinates: nbnd, nc, nkice, nksnow, nkbio, nkaer, nj, ni
Data variables: (12/26)
    time_bounds  (time, nbnd) object 496B dask.array<chunksize=(1, 2), meta=np.ndarray>
    NCAT         (nc) float64 40B dask.array<chunksize=(5,), meta=np.ndarray>
    VGRDi        (nkice) float64 32B dask.array<chunksize=(4,), meta=np.ndarray>
    VGRDs        (nksnow) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    VGRDb        (nkbio) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    VGRDa        (nkaer) float64 40B dask.array<chunksize=(5,), meta=np.ndarray>
    ...           ...
    snoice       (time, nj, ni) float64 408MB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    fhocn        (time, nj, ni) float64 408MB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    dvidtt       (time, nj, ni) float64 408MB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    dvidtd       (time, nj, ni) float64 408MB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    fsurf_ai     (time, nj, ni) float64 408MB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
    sifb         (time, nj, ni) float64 408MB dask.array<chunksize=(1, 1142, 1440), meta=np.ndarray>
Attributes: (12/13)
    title:               access-om3
    contents:            Diagnostic and Prognostic Variables
    source:              CICE Sea Ice Model, unknown_version_name
    comment:             All years have exactly 365 days
    comment2:            File written on model date 19810102
    comment3:            seconds elapsed into model date:      0
    ...                  ...
    time_axis_position:  middle
    Conventions:         CF-1.8
    external_variables:   TLON TLAT ULON ULAT NLON NLAT ELON ELAT NFSD tmask ...
    history:             This dataset was created on 2025-06-26 at 08:52
    date_created:        2025-06-26 08:52
    io_flavor:           io_pio2 hdf5

In [15]:
    client.close()

Process Dask Worker process (from Nanny):
2025-06-27 13:31:55,768 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2025-06-27 13:31:55,769 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2025-06-27 13:31:55,769 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-06-27 13:31:55,769 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2025-06-27 13:31:55,770 - distributed.nanny - ERROR - Worker pro

In [16]:
    #copy back from jobfs and delete
    monthly_f = glob.glob(f"{jobfs}/{MONTHLY_STUB_FN}????-??.nc")

    for iF in monthly_f:
        shutil.copy(iF, "/g/data/tm70/as2285/concat_test/")

    #how to test for success?
    daily_f = glob.glob(f"{in_dir}/{CICE_DAILY_FN}")

    # for iF in daily_f:
        # shutil.rm(iF, jobfs)